In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [27]:
import seaborn as sns

In [28]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [29]:
my_session.connection.schema

'CPG_BRONZE'

In [30]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [31]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [32]:
type(sf_df)

snowflake.snowpark.dataframe.DataFrame

In [33]:
df=sf_df.to_pandas()

In [34]:
df.head()

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202312,2023-12-26,2023-11-27,372.73,2,0.000650,372.727273,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0118,Soap,Medicated Soap,Burgundy
1,202312,2023-12-26,2023-11-27,436.36,3,0.000900,436.363636,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0131,Soap,Toilet Soap,Grape vine
2,202312,2023-12-24,2023-11-27,202.82,2,0.000214,209.090909,202212,DB0209,OL112555,Shelbyville,Tennessee,City Center,Str5,PRD0065,Hair Care,Hair Oil,Crimson
3,202312,2023-12-29,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0655,OL113173,Renton,Washington,Orange,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
4,202312,2023-12-17,2023-11-27,145.45,1,0.000300,145.454545,202212,DB0652,OL112373,Wallingford,Connecticut,Dolphin,Str1,PRD0070,Perfume and Deodrants,Female Deodrant,Pista green


In [35]:
df.shape

(966400, 18)

In [36]:
df.isnull().sum()

MNTH_CODE           0
TRANS_DATE          0
START_DATE          0
SALES_VALUE         0
SALES_UNITS         0
SALES_VOLUME        0
SALES_PTR_VALUE     0
OC_CODE             0
DISTRIBUTOR_CODE    0
OUTLET_CODE         0
CITY                0
STATE               0
COUNTY              0
STREET              0
PRODUCT_CODE        0
CATEGORY            0
SUBCATEGORY         0
BRAND               0
dtype: int64

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 966400 entries, 0 to 966399
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   MNTH_CODE         966400 non-null  int32  
 1   TRANS_DATE        966400 non-null  object 
 2   START_DATE        966400 non-null  object 
 3   SALES_VALUE       966400 non-null  float64
 4   SALES_UNITS       966400 non-null  int16  
 5   SALES_VOLUME      966400 non-null  float64
 6   SALES_PTR_VALUE   966400 non-null  float64
 7   OC_CODE           966400 non-null  int32  
 8   DISTRIBUTOR_CODE  966400 non-null  object 
 9   OUTLET_CODE       966400 non-null  object 
 10  CITY              966400 non-null  object 
 11  STATE             966400 non-null  object 
 12  COUNTY            966400 non-null  object 
 13  STREET            966400 non-null  object 
 14  PRODUCT_CODE      966400 non-null  object 
 15  CATEGORY          966400 non-null  object 
 16  SUBCATEGORY       96

In [38]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
labelencoder = LabelEncoder()

In [39]:
df.duplicated().sum()

3158

In [54]:
df.drop_duplicates(subset='OUTLET_CODE',keep='first')

,MNTH_CODE,TRANS_DATE,START_DATE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,OC_CODE,DISTRIBUTOR_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,PRODUCT_CODE,CATEGORY,SUBCATEGORY,BRAND
0,202312,2023-12-26,2023-11-27,372.73,2,0.000650,372.727273,202212,DB0110,OL112848,Carbondale,Illinois,City Center,Str2,PRD0118,Soap,Medicated Soap,Burgundy
2,202312,2023-12-24,2023-11-27,202.82,2,0.000214,209.090909,202212,DB0209,OL112555,Shelbyville,Tennessee,City Center,Str5,PRD0065,Hair Care,Hair Oil,Crimson
3,202312,2023-12-29,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0655,OL113173,Renton,Washington,Orange,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
4,202312,2023-12-17,2023-11-27,145.45,1,0.000300,145.454545,202212,DB0652,OL112373,Wallingford,Connecticut,Dolphin,Str1,PRD0070,Perfume and Deodrants,Female Deodrant,Pista green
6,202312,2023-12-16,2023-11-27,200.00,2,0.000500,200.000000,202212,DB0110,OL113059,Newburyport,Massachusetts,Sumter,Str2,PRD0096,Lotion,Head Lotion,Fuchsia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964500,202312,2023-12-30,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0110,OL33631,Dillon,Montana,Sumter,Str2,PRD0106,Hair Care,Shampoo,Hair & Shoulders
964541,202312,2023-12-26,2023-11-27,142.86,16,0.000384,142.857143,202212,DB0209,OL33641,Layton,Utah,Sumter,Str2,PRD0112,Soap,Toilet Soap,Lemon yellow
964626,202312,2023-12-17,2023-11-27,140.00,16,0.000368,142.857143,202212,DB0110,OL33633,Woodward,Oklahoma,Sumter,Str1,PRD0116,Soap,Soap Gels,Lime Green
964882,202312,2023-12-29,2023-11-27,154.55,1,0.000240,154.545455,202212,DB0655,OL33643,Canton,Mississippi,Sumter,Str2,PRD0082,Perfume and Deodrants,Unisex Deodrant,Jet black


In [55]:
df.shape

(966400, 18)

In [56]:
Top10products = df.groupby('CATEGORY',as_index=False)['SALES_VOLUME'].sum().sort_values('SALES_VOLUME',ascending=False).head(10)
ax = Top10products.plot(x="CATEGORY",
                   y="SALES_VOLUME", 
                   legend=False,
                   kind='bar',
                   color="green",
                   figsize= (15,6))

plt.grid(False)


plt.xlabel('CATEGORY',fontsize=18)
plt.ylabel('SALES_VOLUME', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.title('Top 10 best-selling Product',Fontsize = 18)

plt.show()

AttributeError: Text.set() got an unexpected keyword argument 'Fontsize'